In [ ]:
import pandas as pd

In [ ]:
import mod as lp

In [ ]:
!pip install xlrd >= 2.0.1

In [ ]:
은행거래 = lp.list_df('../sample/', 'xls')

In [ ]:
세금계산서 = lp.list_df('../sample/', 'xlsx')

In [ ]:
은행거래.head()

In [ ]:
세금계산서.head()

### merge()
1. 2개의 데이터프레임 결합하는 함수
2. 특정한 조건에 맞춰서 데이터프레임을 결합하는 함수
3. on 매개변수는 2개의 데이터프레임에 조건 
4. how 매개변수는 기준 데이터프레임 지정

In [ ]:
df1 = 세금계산서.groupby(['상호', '대표자명']).sum().reset_index()

In [ ]:
df2 = 은행거래.groupby('보낸분/받는분').sum()[['입금액(원)']].reset_index()

In [ ]:
# df2의 컬럼의 이름을 [대표자명, 입금액]로 변경
df2.columns = ['대표자명', '입금액']

In [ ]:
df2

In [39]:
# total_df는 세금계산서와 은행거래를 join 결합
total_df = pd.merge(left = df1, right=df2, on='대표자명', how='left')

In [ ]:
# 결측치 확인
total_df.info()

In [43]:
# 차액 파생변수를 생성 
# 합계금액 - 입금액
total_df['차액'] = total_df['합계금액'] - total_df['입금액']

In [47]:
total_df['차액'].value_counts()

차액
0.0          17
220000.0      2
742500.0      2
495000.0      2
3580500.0     1
             ..
1622500.0     1
4780688.0     1
452262.0      1
5012096.0     1
5784440.0     1
Name: count, Length: 72, dtype: int64

In [56]:
# 미수여부
# 입금완료 -> 차액이 0이하인 경우 
# 미수 -> 차액이 0초과인 경우
total_df.loc[total_df['차액'] > 0, '미수여부'] = '미수'
total_df.loc[total_df['차액'] <= 0, '미수여부'] = '입금완료'

In [57]:
total_df['미수여부'].value_counts()

미수여부
미수      52
입금완료    39
Name: count, dtype: int64

In [62]:
# 데이터프레임을 엑셀로 저장
total_df.to_excel('result.xlsx', index=False)